In [27]:
using Plots
using HDF5
using Unitful
using UnitfulRecipes
using Statistics
using Distributions
using Plots.PlotMeasures

# Load Filter output file

In [39]:
timestamp = ["t0000","t0100", "t0200","t0300","t0400","t0500","t0600","t0700","t0800","t0900","t1000", "t1100", "t1200","t1300","t1400","t1500","t1600"] # Edit this value to plot a different time slice from the list above
field = "ps" # Choose from the fields listed above
output_folder = "uniform/"
unit_comp = 0.01
level = 1

truth_filename = ""
filename = joinpath(output_folder,"")
plot_folder = ""
#Loop through all the gauges and produce the distributions
fh, x_st, y_st = read_hdf5_file(filename, timestamp[1], field, level)
gauges_lon = ustrip(x_st)
gauges_lat = ustrip(y_st)
state_lon = range(0,180,48)
state_lat = range(0,360,96)
for time in timestamp
    @show time
    fh, x_st, y_st = read_hdf5_file(filename, time, field, level)
    z_truth = read_hdf5_truth(truth_filename, time, field, unit_comp, level)
    for gauge_id = 1:50
        pointx, pointy = find_gauge_location(gauge_id, gauges_lon, gauges_lat, state_lon, state_lat)
        particles = loop_over_particles(fh, time, field, unit_comp, pointx, pointy)
        plotting(particles, z_truth, gauge_id, pointx, pointy, state_lon, state_lat, time, plot_folder)
    end
end

time = "t0000"
(gauge_id, timestamp) = 

(2, "t0000")
(gauge_id, timestamp) = (4, "t0000")
(gauge_id, timestamp) = 

(5, "t0000")


(gauge_id, timestamp) = (10, "t0000")
(gauge_id, timestamp) = (12, "t0000")


(gauge_id, timestamp) = (13, "t0000")
(gauge_id, timestamp) = 

...

time = "t1200"
time = 

"t1300"


time = "t1400"
time = 

"t1500"


time = "t1600"


In [28]:

function read_hdf5_file(filename, timestamp, field, level)
    fh = h5open(filename, "r")
    # println("The following datasets found in file ", filename, ": ", keys(fh))
    x_st_unit = read(fh["station_coordinates"]["lon"]["Unit"])
    y_st_unit = read(fh["station_coordinates"]["lat"]["Unit"])
    x_st = read(fh["station_coordinates"]["lon"]) .* uparse(x_st_unit) .|> u"°"
    y_st = read(fh["station_coordinates"]["lat"]) .* uparse(y_st_unit) .|> u"°"
    return fh, x_st, y_st
end

read_hdf5_file (generic function with 1 method)

In [29]:
function read_hdf5_truth(filename, timestamp, field, unit_comp, level)
    th = h5open(filename, "r")
    field_unit = "hPa"
    field_desc = read(th["data_nature"][timestamp][field]["Description"]);
    z_truth = read(th["data_nature"][timestamp][field]) .* uparse(field_unit);
    return z_truth[:,:,level].*unit_comp
end

read_hdf5_truth (generic function with 1 method)

In [30]:
function find_gauge_location(gauge_id, gauges_lon, gauges_lat, state_lon, state_lat)
    pointx = findmin((abs.(state_lon .- ustrip(gauges_lon[gauge_id]))))[2]
    pointy = findmin((abs.(state_lat .- ustrip(gauges_lat[gauge_id]))))[2]
    return pointx, pointy
end

find_gauge_location (generic function with 1 method)

In [31]:
function loop_over_particles(fh, timestamp, field, unit_comp, pointx, pointy)
    field_unit = read(fh["state_particle_1"][timestamp][field]["Unit"])
    field_desc = read(fh["state_particle_1"][timestamp][field]["Description"])
    psA = Float64[]
    for it = 1:256
        particle = read(fh[string("state_particle_",string(it))][timestamp][field]);
        push!(psA, particle[pointx,pointy,level]*unit_comp)
    end
    return psA
end

loop_over_particles (generic function with 1 method)

In [38]:
function plotting(particles, z_truth, gauge_id, pointx, pointy, state_lon, state_lat, timestamp, output_folder)
    distA = Normal(mean(particles),std(particles))
    targetx = z_truth[pointx,pointy]
    skew, kurt = round(skewness(particles), digits = 3), round(kurtosis(particles), digits = 3)
    longitude = round(state_lon[pointx], digits = 2)
    latitude = round(state_lat[pointy], digits = 2)
    histogram(particles, xlabel="Surface Pressure", label="", linecolor = "white", normalize = true, top_margin = 12px, title="Histogram of Surface Pressure at ($longitude °, $latitude °) \n Skewness = $skew  Kurtosis = $kurt")
    vline!([targetx], seriestype = :scatter, color=:red, label="True Value")
    plot!(x->pdf(distA, x), label="Fitted Gaussian", bottom_margin = 12px)
    plot!(size=(1000,400))
    vline!([mean(particles)], seriestype = :scatter, color=:black, label="Mean Value")
    savefig(joinpath(output_folder,timestamp,string(gauge_id)))
    if abs(skew) > 0.5 || abs(kurt) > 1.0
        @show gauge_id, timestamp
    end
    # skewness(psA), kurtosis(psA)
end

plotting (generic function with 1 method)

In [ ]:
using StatsBase
using Random
Random.seed!(1234)
xsamp = rand(distC,size(psC))
# skewness(xsamp), kurtosis(xsamp)
histogram(xsamp.+20)
histogram!(psC.+20)
kldivergence(sort(xsamp.+20),sort(psC.+20), mean(psC)+20)